In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [ ]:
!unzip /content/corn

**ImageDataGenerator options**


featurewise_center=False,
samplewise_center=False,
featurewise_std_normalization=False,
samplewise_std_normalization=False,
zca_whitening=False,
zca_epsilon=1e-06,
rotation_range=0,
width_shift_range=0.0,
height_shift_range=0.0,
brightness_range=None,
shear_range=0.0,
zoom_range=0.0,
channel_shift_range=0.0,
fill_mode='nearest',
cval=0.0,
horizontal_flip=False,
vertical_flip=False,
rescale=None,
preprocessing_function=None,
data_format=None,
validation_split=0.0,
interpolation_order=1,
dtype=None




    featurewise_center: Boolean. Set to True if you want to subtract the mean of the whole dataset from each feature.
    samplewise_center: Boolean. Set to True if you want to subtract the mean of each individual sample/image from itself.
    featurewise_std_normalization: Boolean. Set to True to divide inputs by the standard deviation of the whole dataset.
    samplewise_std_normalization: Boolean. Set to True to divide each individual sample/image by its own standard deviation.
    zca_whitening: Boolean. Set to True to apply ZCA whitening, which helps decorrelate features in the input data.
    zca_epsilon: Float. Epsilon for ZCA whitening, usually set to a small value like 1e-06.
    rotation_range: Int. Range for random rotations in degrees (e.g., 40 means rotation between -40 and +40 degrees).
    width_shift_range: Float or Tuple. Range for horizontal random shifts as a fraction of the width or as an absolute number of pixels.
    height_shift_range: Float or Tuple. Range for vertical random shifts as a fraction of the height or as an absolute number of pixels.
    brightness_range: Tuple. Range for random brightness augmentation.
    shear_range: Float. Range for random shearing in degrees.
    zoom_range: Float or Tuple. Range for random zooming (e.g., 0.2 means zoom between 0.8 and 1.2).
    channel_shift_range: Float. Range for random channel shifts.
    fill_mode: String. How to fill the newly created pixels after rotation or shifting. Options: 'nearest', 'constant', 'reflect', 'wrap'.
    cval: Float. Value used for points outside the boundaries when fill_mode is 'constant'.
    horizontal_flip: Boolean. Set to True for random horizontal flipping.
    vertical_flip: Boolean. Set to True for random vertical flipping.
    rescale: Float. Rescaling factor applied to the input data.
    preprocessing_function: Function. Custom function for data preprocessing.
    data_format: String. Image data format ('channels_last' or 'channels_first').
    validation_split: Float. Fraction of images to reserve for validation if using validation_split with ImageDataGenerator.
    interpolation_order: Int. Interpolation order used in image transformations.
    dtype: Data type to use for the generated arrays.

The selection and recommended values of these augmentation techniques depend on the specific dataset, the nature of the images, and the problem you are trying to solve. For example:

    Use featurewise_center, featurewise_std_normalization, or zca_whitening when you want to normalize the entire dataset to have zero mean or uncorrelated features.
    Use horizontal_flip and/or vertical_flip when the images are not direction-sensitive (e.g., many natural scenes).
    Use rotation_range, width_shift_range, height_shift_range, and zoom_range to introduce more variations in the dataset.
    The value for each parameter can vary based on the dataset and the level of augmentation you want. It's often best to experiment with different values and observe the effects on the performance of your model.

The choice of data augmentation techniques should be guided by the nature of the data and the potential benefits they can bring to the model's generalization ability. You may want to start with a minimal set of augmentations and gradually introduce more if necessary. Always monitor the performance of your model on a validation set to avoid overfitting.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(
    rescale=1./255
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/content/corn/train',  # This is the source directory for training images
        target_size=(224, 224),  # All images will be resized to 150x150
        class_mode='categorical')

# Flow training images in batches of 128 using train_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        '/content/corn/val',  # This is the source directory for training images
        target_size=(224, 224),  # All images will be resized to 150x150
        class_mode='categorical')

In [ ]:
model = tf.keras.applications.VGG19(weights = "imagenet", include_top=False, input_shape = (224, 224, 3))
#Adding custom Layers
x = model.output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
predictions = tf.keras.layers.Dense(4, activation="softmax")(x)
model = tf.keras.Model(inputs = model.input, outputs = predictions)
model.summary()
model.compile(optimizer='adam',
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [ ]:
model_history = model.fit(
train_generator,
epochs = 30,
validation_data = validation_generator)

In [ ]:
import matplotlib.pyplot as plt
acc = model_history.history['accuracy']
val_acc = model_history.history['val_accuracy']
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Val accuracy')
plt.title('Training Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Val Loss')
plt.title('Training Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.show()